In [61]:
import re

import requests
import petl as etl
from bs4 import BeautifulSoup

from pipeline_utils.filesystem.paths import REF_DATA

In [3]:
with requests.get('https://resources.companieshouse.gov.uk/sic/') as r:
    doc = BeautifulSoup(r.text)

In [63]:
sic_ref = etl.fromcolumns(
    [
        [re.sub(r'\n', ' - ', r.text.strip()) for r in doc.table.tbody.find_all('tr')]
    ],
    header=['section']
).addfield(
    'code', lambda r: r.section
).update(
    'code', None, where=lambda r: r.code.startswith('S')
).update(
    'section', None, where=lambda r: r.code is not None
).filldown(
    'section'
).selectnotnone(
    'code'
).convertall(
    lambda c: c.split(' - ')
).unpack(
    'section', newfields = ['section', 'section_name']
).unpack(
    'code', newfields = ['code', 'name']
).convertall(
    lambda v: v.strip()
)

In [65]:
sic_ref.tocsv(REF_DATA / 'sic-lookup.csv')